<h1>Custom Classifiers with IBM Watson Visual Recognition in Python</h1>

<h2>Introduction</h2>

<p><b>Welcome!</b> This notebook is how to operate the Watson Visual Recognition API using the Python Programming Language. Here, I can automate the training and test the Visual Recognition model.</p>

<p>This classifies images of different kinds of furnitures viz. chairs, beds and tables by running python code.</p>

In [1]:
!pip install --upgrade ibm-watson

     |████████████████████████████████| 378kB 8.2MB/s eta 0:00:01
     |████████████████████████████████| 204kB 32.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/71/9a/0a/9b3ca8eca69bc5362eb04709a750b30055a9d27818fd0c9494
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


In [2]:
# Paste API key for IBM Watson Visual Recognition below:
my_apikey = '0000000000000'

In [3]:
from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator(my_apikey)

visrec = VisualRecognitionV3('2018-03-19', 
                             authenticator=authenticator)

In [4]:
import json
with open('chairs.zip', 'rb') as chairs, \
     open('tables.zip', 'rb') as tables, \
     open('beds.zip', 'rb') as beds:
        response = visrec.create_classifier(name="apartmentclassifier",
                                        positive_examples={'chairs': chairs, \
                                                           'tables': tables, \
                                                           'beds': beds})
print(json.dumps(response.get_result(), indent=2))

{
  "classifier_id": "apartmentclassifier_145929029",
  "name": "apartmentclassifier",
  "status": "training",
  "owner": "9014ff07-157d-4279-b301-fde48d8dd228",
  "created": "2020-08-01T22:17:42.101Z",
  "updated": "2020-08-01T22:17:42.101Z",
  "classes": [
    {
      "class": "beds"
    },
    {
      "class": "tables"
    },
    {
      "class": "chairs"
    }
  ],
  "rscnn_enabled": false,
  "core_ml_enabled": true
}


In [5]:
#lets grab the classifier id
classifier_id = response.get_result()["classifier_id"]
classifier_id

'apartmentclassifier_145929029'

In [9]:
Status = visrec.get_classifier(classifier_id=classifier_id, verbose=True).get_result()['status']
if Status=='training': 
    print ("Please, Wait to complete training.......")
else:
    print("Good to go ")

Good to go 


<a id="ref4"></a>
<h2>Testing Classifier</h2>
<p>Let's test the classifier, the function <b>getdf_visrec</b> below uses the method <b>classify</b> from Watson Visual Recognition API to upload the image to the classifier and give us a result in JSON(JavaScript Object Notation) format. Then we use the method <b>json_normalize</b> from the "Pandas" library in Python to turn the result into a table because it is more human readable.</p>

In [10]:
from pandas.io.json import json_normalize

def getdf_visrec(url, classifier_ids, apikey = my_apikey):
    
    json_result = visrec.classify(url=url,
                              threshold='0.6',
                              classifier_ids=classifier_id).get_result()
    
    json_classes = json_result['images'][0]['classifiers'][0]['classes']
    
    df = json_normalize(json_classes).sort_values('score', ascending=False).reset_index(drop=True)
    
    return df

<img src="https://m.media-amazon.com/images/I/41rvIBzrjVL._SR500,500_.jpg">
<p>Let's test the Visual Recognition model on this picture of a chair</p>

In [13]:
getdf_visrec(url = 'https://m.media-amazon.com/images/I/41rvIBzrjVL._SR500,500_.jpg',
            classifier_ids=classifier_id)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


,class,score
0,chairs,0.873


<img src="https://www.abc.net.au/cm/rimage/7344430-4x3-xlarge.jpg?v=2">
<p>Let's test the Visual Recognition model on this picture of a bed</p>

In [14]:
getdf_visrec(url = 'https://www.abc.net.au/cm/rimage/7344430-4x3-xlarge.jpg?v=2',
            classifier_ids=classifier_id)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


,class,score
0,beds,0.906


<img src="https://images-na.ssl-images-amazon.com/images/I/71jITyNLgYL._AC_SL1500_.jpg">
<p>Let's test the Visual Recognition model on this picture of a table</p>

In [15]:
getdf_visrec(url = 'https://images-na.ssl-images-amazon.com/images/I/71jITyNLgYL._AC_SL1500_.jpg',
            classifier_ids=classifier_id)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # This is added back by InteractiveShellApp.init_path()


,class,score
0,tables,0.908
